<a href="https://colab.research.google.com/github/Adil1979/P6_CentraleSupelec_OCR/blob/master/02_P6_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projet6: Catégorisation automatique de questions**

## **1.Librairies**

In [ ]:
!pip install feature_engine

In [ ]:
!pip install plotly

In [ ]:
!pip install chart-studio

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.lines import Line2D
import matplotlib as mpl
import chart_studio.plotly.plotly as py
#import plotly.plotly as py
import seaborn as sns
import datetime as dt
import calendar as cld
from scipy import stats
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from os import listdir
import glob
import missingno as msno
from feature_engine.categorical_encoders import OneHotCategoricalEncoder
from sklearn.decomposition import PCA
from matplotlib import dates
import nltk
import re
from nltk.corpus import stopwords 
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer 
import string
from collections import Counter
from bs4 import BeautifulSoup


SELECT Id, Body, Title, Tags

FROM Posts

WHERE Score >= 3 AND PostTypeId = 1

ORDER BY RAND()
 
ASC OFFSET 0 ROWS FETCH NEXT 50000 ROWS ONLY; de 0 à 50000

## **2.Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df_analysis_01 = pd.read_csv('/content/drive/My Drive/CentraleSupelec_OCR/P6/QueryResults_01.csv')
df_analysis_01.head()

In [ ]:
df_analysis_01.shape

In [ ]:
df_analysis_02 = pd.read_csv('/content/drive/My Drive/CentraleSupelec_OCR/P6/QueryResults_02.csv')
df_analysis_02.head()

In [ ]:
df_analysis_02.shape

In [ ]:
df_analysis_03 = pd.read_csv('/content/drive/My Drive/CentraleSupelec_OCR/P6/QueryResults_03.csv')
df_analysis_03.head()

In [ ]:
df_analysis_03.shape

In [ ]:
frames = [df_analysis_01, df_analysis_02, df_analysis_03]

In [ ]:
df_analysis = pd.concat(frames)
df_analysis.head(10)

In [ ]:
df_sample = df_analysis.sample(n=40000)

In [ ]:
df_sample.info()

In [ ]:
df_sample = df_sample.loc[:, ['Id', 'Title', 'Body', 'Tags']]
df_sample = df_sample.set_index('Id')
df_sample.head()

## **3.Preprocessing**

In [ ]:
df_sample.duplicated(subset=["Body", "Title"]).sum()

In [ ]:
example_body = df_sample.Body.iloc[0]

In [ ]:
example_body

In [ ]:
example_title = df_sample.Title.iloc[0]

In [ ]:
example_title

In [ ]:
#Merge Title and Body
df_sample['Title_Body'] = df_sample['Title'] + ' ' + df_sample['Body']

In [ ]:
df_sample.head()


In [ ]:
variables_reorganization = ['Title_Body', 'Tags']
df_sample = df_sample[['Title_Body', 'Tags']]

In [ ]:
df_sample.head()

In [ ]:
df_sample.Title_Body.iloc[0]

In [ ]:
#HTML characters removing
def remove_html(body):
  html_regex = re.compile('<.*?>') #Compile regular expresions
  return re.sub(html_regex, ' ', str(body)) # Replace regex by ' '

In [ ]:
df_sample['Title_Body'] = df_sample['Title_Body'].apply(remove_html)
df_sample.head()

In [ ]:
df_sample.Title_Body.iloc[0]

In [ ]:
#URL removing
def remove_url(body):
  url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
  return re.sub(url_regex, ' ', str(body))  

In [ ]:
df_sample['Title_Body'] = df_sample['Title_Body'].apply(remove_url)
df_sample.head()

In [ ]:
df_sample.Title_Body.iloc[0]

In [ ]:
#Punctuation removing
def remove_punc(body):
  clean_text = re.sub(r'[?|!|"|:|=|_|{|}|[|]|-|$|%|^|&|]',r' ',str(body))
  clean_text = re.sub(r'[.|,|)|(|\|/|-|~|`|>|<|*|$|@|;|→]',r' ', clean_text)
  return clean_text

In [ ]:
df_sample['Title_Body'] = df_sample['Title_Body'].apply(remove_punc)

In [ ]:
df_sample.Title_Body.iloc[0]

In [ ]:
#All characters removing except letters, numbers 
#Lower
def remove_other(body):
  text = str(body)
  clean_text = re.sub(r"[^a-zA-Z0-9#+-]", " ", text.lower())
  return clean_text

In [ ]:
df_sample['Title_Body'] = df_sample['Title_Body'].apply(remove_other)
df_sample.head()

In [ ]:
df_sample.Title_Body.iloc[0]

In [ ]:
#Space removing
def remove_space(body):
  return ' '.join(str(body).split()) 

In [ ]:
df_sample['Title_Body'] = df_sample['Title_Body'].apply(remove_space)

In [ ]:
df_sample.Title_Body.iloc[0]

In [ ]:
#df_analysis['Title_Body_count'] = df_analysis['Title_Body'].apply(lambda text: len(text.split(" ")))

In [ ]:
df_sample.head()

In [ ]:
#Tokenization
nltk.download("punkt")
from nltk.tokenize import word_tokenize

In [ ]:
stop_words = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer().lemmatize
ps = PorterStemmer()

In [ ]:
def remove_encoding_text(text):
    text = str(text)
    text = " ".join(word for word in text.split() if word not in stop_words)
    return text

In [ ]:
df_sample['Title_Body'] = df_sample['Title_Body'].apply(remove_encoding_text)

In [ ]:
df_sample.Title_Body.iloc[0]

In [ ]:
df_sample.head()

In [ ]:
def tokenize(document):
  tokens = [lemma(w) for w in document.split() if w.isalpha()]
  return tokens

In [ ]:
df_sample['Title_Body'].dtype

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('all')

In [ ]:
vectorizer = TfidfVectorizer(tokenizer = tokenize, stop_words = stop_words)

In [ ]:
tfidf = vectorizer.fit_transform(df_sample['Title_Body'])

In [280]:
df_sample.Title_Body.iloc[0]

'asp net core web api class library trying make n tier application web api kept different class library made testcontroller controller class different class library code goes like using microsoft aspnetcore mvc using system using system collections generic namespace pkruni sms api apicontroller produces application json route api controller public class testcontroller controllerbase public ienumerable lt string gt get return new string chris hadfield question access web api main project already added reference class library main project working api working 404 page found error shows trying access api project structure wrong please help'

In [ ]:
vectorizer.vocabulary_.items()

In [ ]:
#get word frequencies and create wordcloud
tfidf_weights = [(word, tfidf.getcol(idx).sum()) for word, idx in vectorizer.vocabulary_.items()]

In [ ]:
type(tfidf_weights)

In [ ]:
tfidf_weights[0:10]

In [286]:
from wordcloud import WordCloud

In [ ]:
w = WordCloud(width=1500, height=1200, mode='RGBA', background_color='white', max_words=2000).fit_words(dict(tfidf_weights))

In [ ]:
# all below in a single cell
plt.figure(figsize=(20,15))
plt.imshow(w)
plt.axis('off')
plt.savefig('recipes_wordcloud.png')

In [ ]:
from wordcloud import WordCloud

In [ ]:
df_analysis.Tags.iloc[0]

In [ ]:
# convert data to list
to_list = {'><': "','",'<':"['", '>':"']"}
for key, value in to_list.items():
  df_analysis['Tags'] = df_analysis['Tags'].str.replace(key, value)

In [ ]:
df_analysis.head()

In [ ]:
tags = df_analysis['Tags'].tolist()

In [ ]:
tags

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer # Convert a collection of text documents to a matrix of token counts

In [ ]:
cv = CountVectorizer(max_features = 50) #, analyzer= 'word', stop_words= 'english')
tags_vec = cv.fit_transform(tags).toarray()
print('Number of tags: ', len(cv.vocabulary_))
tags_name = cv.get_feature_names()
tags_count = tags_vec.sum(axis=0)
print('count of Tags: ')
print(dict(zip(tags_name, tags_count)))

In [ ]:
dic_tags = dict(zip(tags_name, tags_count))
dic_tags

In [ ]:
#plt.bar(list(dic_tags.keys()), dic_tags.values(), color='g', width= 0.5)
#plt.show()


In [ ]:
df_tags_count = pd.DataFrame(dic_tags.items(), columns = ['Tag', 'Count']).sort_values(by='Count', ascending = False )

In [ ]:
df_tags_count

In [ ]:
#df_sorted = df_tags_count.sort_values(by='Count', ascending=False)

sns.catplot(x='Tag', y='Count', data=df_tags_count, kind='bar', height=15, aspect=10.7/1.27);


In [ ]:
ax  = sns.countplot(x = 'Tag', data = df_tags_count.head(10))
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.show()

Depuis le départ, on a seulement utilisé les fréquences d'apparition des différents mots/n-grammes présents dans notre corpus. Le problème est que si l'on veut vraiment représenter un document par les n-grammes qu'il contient, il faudrait le faire relativement à leur apparition dans les autres documents.

En effet, si un mot apparait dans d'autres documents, il est donc moins représentatif du document qu'un mot qui n'apparait que uniquement dans ce document.

I will be covering these topics in the project, but you can go through them before or during the project.

1. http://www.tfidf.com/ - Gives the formula for TF-IDF calculation and a simple example.

2. Difference between stemming and lemmatizing -

The goal of both stemming and lemmatization is to reduce inflectional forms and sometimes derivationally related forms of a word to a common base form. However, the two words differ in their flavor. Stemming usually refers to a crude heuristic process that chops off the ends of words in the hope of achieving this goal correctly most of the time, and often includes the removal of derivational affixes. Lemmatization usually refers to doing things properly with the use of a vocabulary and morphological analysis of words, normally aiming to remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma.

## **TF-IDF**

Depuis le départ, on a seulement utilisé les fréquences d'apparition des différents mots/n-grammes présents dans notre corpus. Le problème est que si l'on veut vraiment représenter un document par les n-grammes qu'il contient, il faudrait le faire relativement à leur apparition dans les autres documents.

En effet, si un mot apparait dans d'autres documents, il est donc moins représentatif du document qu'un mot qui n'apparait que uniquement dans ce document.

I will be covering these topics in the project, but you can go through them before or during the project.

http://www.tfidf.com/ - Gives the formula for TF-IDF calculation and a simple example.

Difference between stemming and lemmatizing -

The goal of both stemming and lemmatization is to reduce inflectional forms and sometimes derivationally related forms of a word to a common base form. However, the two words differ in their flavor. Stemming usually refers to a crude heuristic process that chops off the ends of words in the hope of achieving this goal correctly most of the time, and often includes the removal of derivational affixes. Lemmatization usually refers to doing things properly with the use of a vocabulary and morphological analysis of words, normally aiming to remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma.